In [2]:
import pandas as pd
import re

In [3]:
# Read the data.
df = pd.read_csv('NotgeldData.csv')
# Capitalize the first letter of each column.
df.columns = [col.title() for col in df.columns]
# Drop Katalog, Url and Date.
df.drop(['Katalog', 'Url', 'Date'], axis=1, inplace=True)

In [649]:
# Clean the Ort.
# Split the Title column on the comma delimiter
df[['FirstPart', 'SecondPart']] = df['Title'].str.split(',', n=1, expand=True)

# check if the FirstPart is in the Ort column
mask = df['FirstPart'].isin(df['Ort'])

# remove the FirstPart from the Title column where the mask is True
df.loc[mask, 'Title'] = df['SecondPart']

# drop the FirstPart and SecondPart columns
df = df.drop(columns=['FirstPart', 'SecondPart'])
# Save the updated DataFrame to a new file
df.to_csv('Updated.csv', index=False)

In [650]:
# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Split the string using the comma delimiter
    parts = row['Title'].split(', ')
    # Check if the first part of the split string contains any digits
    if any(char.isdigit() for char in parts[0]):
        # Set issuer value to None
        issuer = None
    else:
        # Set issuer value to the first part of the split string
        issuer = parts[0]
    # Update the issuer column with the issuer value
    df.at[index, 'Issuer'] = issuer
# Delete the space if the column starts with a space.
df['Issuer'] = df['Issuer'].str.lstrip()
# Fill Nan values with 'NoIssuerInfo'
df['Issuer'].fillna('NoIssuerInfo', inplace=True)
# Save the updated DataFrame to a new file
df.to_csv('Updated.csv', index=False)

In [651]:
# Clean the Issuer.
# Loop through the rows of the DataFrame
for index, row in df.iterrows():
    # Check if the Wert value is in the Title string
    if row['Issuer'] in row['Title']:
        # Remove the Wert value from the Title string
        df.at[index, 'Title'] = row['Title'].replace(row['Issuer'], '')
# Save the updated DataFrame to a new file
df.to_csv('Updated.csv', index=False)

In [652]:
# Clean the Wert.
# Loop through the rows of the DataFrame
for index, row in df.iterrows():
    # Check if the Wert value is in the Title string
    if row['Wert'] in row['Title']:
        # Remove the Wert value from the Title string
        df.at[index, 'Title'] = row['Title'].replace(row['Wert'], '')

# Save the updated DataFrame to a new file
df.to_csv('Updated.csv', index=False)

In [653]:
# Define a regular expression pattern to match dates in the format "d.m.yy" or "dd.mm.yy"
pattern = r'\b(\d{1,2}\.\d{1,2}\.\d{2})\b'

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Search for dates in the string using the regular expression pattern
    match = re.search(pattern, row['Title'])
    if match:
        # Extract the matched date string
        date_str = match.group(1)
        # Update the corresponding column with the raw date string
        df.at[index, 'Date'] = date_str
    elif 'oD' in row['Title']:
        # Update the corresponding column with "oD"
        df.at[index, 'Date'] = "oD"
    else:
        # Update the corresponding column with "no date"
        df.at[index, 'Date'] = "NoDate"

# Save the updated DataFrame to a new file
df.to_csv('Updated.csv', index=False)

In [654]:
# Clean the Date.
# Loop through the rows of the DataFrame
for index, row in df.iterrows():
    # Check if the Wert value is in the Title string
    if row['Date'] in row['Title']:
        # Remove the Wert value from the Title string
        df.at[index, 'Title'] = row['Title'].replace(row['Date'], '')

# Save the updated DataFrame to a new file
df.to_csv('Updated.csv', index=False)

In [655]:
# Sort the data frame columns in the desired order.
# Ort, Provinz, Wert, Periode, Erhaltung, issuer, date, title
df = df[['Ort', 'Provinz', 'Wert', 'Date', 'Periode', 'Price',  'Erhaltung', 'Issuer',  'Title']]
# Save the updated DataFrame to a new file
df.to_csv('Updated.csv', index=False)

In [656]:
# Save it as an excel file.
df.to_excel('Notgeld_V2.xlsx', index=False)